<h2>Scrape and Imports</h2>

In [1]:
# All the imports

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium 

Montreal raw Data

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# The code was removed by Watson Studio for sharing.

,no_qr,Neighbourhood,no_arr,Borough,nom_mun
86,72,Pierrefonds-Est,31.0,Pierrefonds-Roxboro,Montréal
87,75,L'Île-Bizard–Sainte-Geneviève,32.0,L'Île-Bizard–Sainte-Geneviève,Montréal
88,70,Vieux-Lachine - Saint-Pierre,27.0,Lachine,Montréal
89,60A,Ouest,16.0,Montréal-Nord,Montréal
90,60B,Centre-Nord,16.0,Montréal-Nord,Montréal


In [4]:
mount = df_data_3.drop(columns=['no_qr', 'nom_mun','no_arr'])


In [5]:

body = client_55e1b85e93954a0a8adb3f781687106a.get_object(Bucket='capstone-donotdelete-pr-zvspxpvrshowno',Key='Neighbourhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_5 = pd.read_csv(body)
df_data_5.head()


,Neighbourhood,Latitude,Longitude
0,Rivière-des-Prairies,45.6677,-73.5462
1,Beaurivage,45.5982,-73.5139
2,Tétreaultville,45.6018,-73.5246
3,Dupéré,45.6046,-73.5357
4,Guybourg,45.5742,-73.5206


Time to merge!

In [6]:
# Combine the two data sets
city = mount.merge(df_data_5, how='inner', on='Neighbourhood')
city

,Neighbourhood,Borough,Latitude,Longitude
0,Rivière-des-Prairies,Rivière-des-Prairies–Pointe-aux-Trembles,45.66770,-73.54620
1,Beaurivage,Mercier–Hochelaga-Maisonneuve,45.59820,-73.51390
2,Tétreaultville,Mercier–Hochelaga-Maisonneuve,45.60180,-73.52460
3,Dupéré,Mercier–Hochelaga-Maisonneuve,45.60460,-73.53570
4,Guybourg,Mercier–Hochelaga-Maisonneuve,45.57420,-73.52060
5,Longue-Pointe,Mercier–Hochelaga-Maisonneuve,45.57810,-73.53820
6,Louis-Riel,Mercier–Hochelaga-Maisonneuve,45.58530,-73.55330
7,Marie-Victorin,Rosemont–La Petite-Patrie,45.56830,-73.56810
8,Maisonneuve,Mercier–Hochelaga-Maisonneuve,45.55760,-73.53660
9,René-Goupil,Villeray–Saint-Michel–Parc-Extension,45.57850,-73.61470


In [7]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(city['Borough'].unique()),
        city.shape[0]
    )
)

The dataframe has 33 boroughs and 91 neighbourhoods.


In [8]:
address = 'Montreal City, QC' 

geolocator = Nominatim(user_agent="qb_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


In [9]:
# create map of Quebec City using latitude and longitude values
map_quebec = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(city['Latitude'], city['Longitude'], city['Borough'], city['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quebec)  
    
map_quebec

<h2>Finished with the Data. Setting up the maps!</h2>

In [10]:
# Foursqaure ID

import json
{
    "tags": [
        "remove-input",
    ]
}
CLIENT_ID = '4F2TWL40HEUEKJXU01ZUMRE4ZNYOGGTRNMPLGRVNJT33J1KJ' # your Foursquare ID
CLIENT_SECRET = 'O5MCRNOS2HOODSSYWX15PYEVB05WZMPY2VGB3GC1K4KQBCFG' # your Foursquare Secret
ACCESS_TOKEN = 'KFUMX2PWRS3FVCUZRGFCYPKLJ1VEKWDLC2NWLHOW24M3S5C4'
version = '20180605' # Foursquare API version
limit = 100 # A default Foursquare API limit value
radius = 2000 

First Neighbourhood

Create Foursquare link

In [15]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        categoryId = '4d4b7105d754a06374d81259' # ID of 'Food'
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            categoryId, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'],
            v['id'],
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                    'Venue Id',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
city_venues = getNearbyVenues(names=city['Neighbourhood'],
                                   latitudes=city['Latitude'],
                                   longitudes=city['Longitude']
                                  )

Rivière-des-Prairies


KeyError: 'venues'

In [17]:
cityven = city_venues.drop_duplicates(subset=['Venue','Venue Latitude','Venue Longitude'], keep='last')
cityven.head()

NameError: name 'city_venues' is not defined

In [14]:
print('There are {} uniques categories.'.format(len(cityven['Venue Category'].unique())))

NameError: name 'cityven' is not defined

<h2> What is where.</h2>

In [ ]:
# one hot encoding
city_onehot = pd.get_dummies(cityven[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighbourhood'] = cityven['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

In [ ]:
city_grouped = city_onehot.groupby('Neighbourhood').mean().reset_index()
city_grouped.head()

<h2> The ratings</h2>

In [ ]:
def getVenueRatings(venueId):
    
    venues_list=[]
    like_list=[]
    for venueId in zip(venueId):
        print(venueId)
    
        url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
            venueId, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            version, 
           )    
    
        results = requests.get(url).json()["response"]
        like_list.append((r['likes']['count']) for r in results)
        venues_list.append([(venueId,
            like_list)])
        
        near_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        near_venues.columns = ['Venue Id',  
                  'Venue Likes']
    return(near_venues)

In [ ]:
venuelist = getVenueRatings(venueId=cityven['Venue Id'])

In [ ]:
venuelist

In [ ]:
print(venuelist['Venue Likes'].max())
print(venuelist['Venue Likes'].min())
print(venuelist['Venue Likes'].median())
print(venuelist['Venue Likes'].mean())

In [ ]:
venuelist['Venue Likes'].hist(bins=4)
plt.show()

In [ ]:
# now we have our bin values so let's set them to the appropriate values
# less than 24, 24-45, 45-76, 76>
# poor, below avg, abv avg, great

poor = venuelist['Venue Likes']<=24
below_avg = venuelist[(venuelist['Venue Likes']>24) & (venuelist['Venue Likes']<=45)]
abv_avg = venuelist[(venuelist['Venue Likes']>45) & (venuelist['Venue Likes']<=76)]
great = venuelist['Venue Likes']>76


In [ ]:
def conditions(l):
    if l['Venue Likes']<=24:
        return 'poor'
    if l['Venue Likes']<=45:
        return 'below average'
    if l['Venue Likes']<=76:
        return 'average'
    if l['Venue Likes']>76:
        return 'great'

venuelist['Likes Categories']=to_venues.apply(conditions, axis=1)



<h2>Make a cluster map!</h2>

In [ ]:
kclusters = 5

city_grouped_clustering = city_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
city_merged = city.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [ ]:
# create map
latitude = 45.5017
longitude = -73.5673
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Neighbourhood'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[0] + list(range(5, city_merged.shape[1]))]]


<h2>Chloropleth Map</h2>

In [ ]:
# download countries geojson file
!wget --quiet https://data.montreal.ca/dataset/00bd85eb-23aa-4669-8f1b-ba9a000e3dd8/resource/e9b0f927-8f75-458c-8fda-b5da65cc8b73/download/limadmin.geojson

print('GeoJSON file downloaded!')

In [ ]:
montreal_geo = r'limadmin.geojson' # geojson file

# create a plain world map
world_map = folium.Map(location=[0, 0], zoom_start=2)

In [ ]:
threshold_scale = np.linspace(city_merged['Cluster Labels'].min(),
                              city_merged['Cluster Labels'].max(),
                              10, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[0] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration


latitude = 45.5017
longitude = -73.5673

mount = folium.Map(location=[latitude, longitude], zoom_start=10)

mount.choropleth(
    geo_data=montreal_geo,
    data=city_merged,
    columns=['Neighbourhood','Cluster Labels'],
    key_on='feature.properties.NOM',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.1,
)

# display map
mount